## Şimdi ilk adım olan temel tek-head Cross-Attention’ı kodlayalım ve adım adım açıklayalım.

In [1]:
import torch
import torch.nn.functional as F

def cross_attention(Q, K, V, mask=None):
    """
    Tek-head Cross-Attention
    Q: decoder input → (batch, seq_len_dec, d_k)
    K: encoder output → (batch, seq_len_enc, d_k)
    V: encoder output → (batch, seq_len_enc, d_v)
    mask: opsiyonel (batch, seq_len_enc)
    """
    # Skor hesaplama
    scores = torch.matmul(Q, K.transpose(-2, -1)) / (Q.size(-1) ** 0.5)
    
    # Mask uygulama,
    if mask is not None:
        scores = scores.masked_fill(mask.unsqueeze(1) == 0, float('-inf'))
    
    # Softmax → ağırlıklı toplam
    attn = F.softmax(scores, dim=-1)
    out = torch.matmul(attn, V)
    return out, attn

# Test
B, L_dec, L_enc, d_k, d_v = 2, 4, 6, 8, 8
Q = torch.randn(B, L_dec, d_k)
K = torch.randn(B, L_enc, d_k)
V = torch.randn(B, L_enc, d_v)

out, attn_map = cross_attention(Q, K, V)
print("Output:", out.shape)        # (B, L_dec, d_v)
print("Attention map:", attn_map.shape)  # (B, L_dec, L_enc)


Output: torch.Size([2, 4, 8])
Attention map: torch.Size([2, 4, 6])


### Bu noktada Cross-Attention tek-head olarak çalışıyor ve mantığı:

* Decoder token’ları → Q

* Encoder token’ları → K, V

* Skor = QKᵀ / √d_k

* Softmax + context = attn × V

---

## 🔹 Multi-Head Cross-Attention

### Amaç
- Tek bir head sadece bir tür ilişkiyi öğrenebilir.  
- Multi-head ile farklı “bakış açıları” paralel olarak öğrenilir.  
- Decoder token’ları encoder’daki farklı temsillere aynı anda dikkat edebilir.

### Adımlar
1. Q/K/V projeksiyonları → (B, L, E)  
2. Head’lere böl → (B, num_heads, L, head_dim)  
3. Her head için skor → softmax → ağırlıklı toplam  
4. Head’leri birleştir (concat) → lineer çıkış  
5. Opsiyonel: Dropout, Residual, LayerNorm


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Linear projeksiyonlar
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # Dropout + LayerNorm
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, x_dec, x_enc, mask_enc=None):
        B, L_dec, E = x_dec.shape
        L_enc = x_enc.size(1)
        
        # 1️⃣ Q/K/V projeksiyonları
        Q = self.q_proj(x_dec).view(B, L_dec, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(x_enc).view(B, L_enc, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(x_enc).view(B, L_enc, self.num_heads, self.head_dim).transpose(1,2)
        
        # 2️⃣ Skor hesaplama
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Mask
        if mask_enc is not None:
            mask = mask_enc.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # 3️⃣ Softmax ve ağırlıklı toplam
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)  # (B, num_heads, L_dec, head_dim)
        
        # 4️⃣ Head birleştirme
        context = context.transpose(1,2).contiguous().view(B, L_dec, E)
        out = self.layernorm(x_dec + self.dropout(self.out_proj(context)))
        return out, attn

# Test
B, L_dec, L_enc, E = 2, 5, 8, 64
x_dec = torch.randn(B, L_dec, E)
x_enc = torch.randn(B, L_enc, E)
mask_enc = torch.ones(B, L_enc)

cross_attn = MultiHeadCrossAttention(E, num_heads=8)
y, attn_map = cross_attn(x_dec, x_enc, mask_enc)
print("Output:", y.shape)        # (B, L_dec, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L_dec, L_enc)


Output: torch.Size([2, 5, 64])
Attention map: torch.Size([2, 8, 5, 8])


### 💡 Bu noktada:

* Decoder token’ları, encoder bağlamını multi-head ile paralel olarak öğreniyor.

* Residual + LayerNorm + Dropout eklenmiş durumda.

* Padding mask desteği var.

----
## 🔹 İleri Düzey Cross-Attention (Bağımsız Blok)

### Özellikler:
1. **Multi-Head:** Birden fazla head ile paralel dikkat
2. **Residual + LayerNorm:** Stabil öğrenme
3. **Dropout:** Overfitting’i azaltma
4. **Masking:** Padding veya look-ahead maskeleri
5. **Bağımsız kullanım:** Artık encoder-decoder zorunluluğu yok, herhangi iki tensor arasında çalışabilir


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedCrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads=8, dp=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # Q/K/V projeksiyonları
        self.q_proj = nn.Linear(embed_dim, embed_dim)
        self.k_proj = nn.Linear(embed_dim, embed_dim)
        self.v_proj = nn.Linear(embed_dim, embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)
        
        # Dropout ve LayerNorm
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim)
        
    def forward(self, Q_input, KV_input, mask=None, causal=False):
        """
        Q_input: (B, L_q, E) → sorgular
        KV_input: (B, L_kv, E) → anahtar ve değerler
        mask: opsiyonel (B, L_kv)
        causal: opsiyonel, look-ahead mask
        """
        B, L_q, E = Q_input.shape
        L_kv = KV_input.size(1)
        
        # Q/K/V projeksiyonları
        Q = self.q_proj(Q_input).view(B, L_q, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor hesaplama
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Padding mask
        if mask is not None:
            mask = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # Causal mask
        if causal:
            causal_mask = torch.triu(torch.ones(L_q, L_kv, device=Q_input.device), diagonal=1).bool()
            scores = scores.masked_fill(causal_mask, float('-inf'))
        
        # Softmax ve ağırlıklı toplam
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        
        # Head birleştirme ve çıkış
        context = context.transpose(1,2).contiguous().view(B, L_q, E)
        out = self.layernorm(Q_input + self.dropout(self.out_proj(context)))
        return out, attn

# Test
B, L_q, L_kv, E = 2, 5, 7, 64
Q_input = torch.randn(B, L_q, E)
KV_input = torch.randn(B, L_kv, E)
mask = torch.ones(B, L_kv)

adv_cross_attn = AdvancedCrossAttention(E, num_heads=8)
y, attn_map = adv_cross_attn(Q_input, KV_input, mask=mask, causal=True)

print("Output:", y.shape)        # (B, L_q, E)
print("Attention map:", attn_map.shape)  # (B, num_heads, L_q, L_kv)


Output: torch.Size([2, 5, 64])
Attention map: torch.Size([2, 8, 5, 7])


---
### O zaman şimdi AdvancedCrossAttention’ı bir adım daha ileriye taşıyalım:

## Hedefler:

* Dynamic Q/K/V boyutları: Girdiler farklı embedding boyutlarına sahip olabilir, otomatik uyarlansın.

* Opsiyonel Fusion / Gating Mekanizması: Q ve context çıktısı birleştirilirken gelişmiş bir füzyon ile daha zengin temsil sağlansın.

---

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicFusionCrossAttention(nn.Module):
    def __init__(self, embed_dim_q, embed_dim_kv, num_heads=8, dp=0.1):
        super().__init__()
        # Multi-head parametreleri
        self.num_heads = num_heads
        self.head_dim = embed_dim_q // num_heads
        
        # Linear projeksiyonlar
        self.q_proj = nn.Linear(embed_dim_q, embed_dim_q)
        self.k_proj = nn.Linear(embed_dim_kv, embed_dim_q)  # farklı embed_dim support
        self.v_proj = nn.Linear(embed_dim_kv, embed_dim_q)
        self.out_proj = nn.Linear(embed_dim_q, embed_dim_q)
        
        # Gating mekanizması
        self.gate_proj = nn.Linear(2*embed_dim_q, embed_dim_q)
        
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim_q)
        
    def forward(self, Q_input, KV_input, mask=None, causal=False):
        B, L_q, E_q = Q_input.shape
        L_kv = KV_input.size(1)
        
        # Q/K/V projeksiyonları
        Q = self.q_proj(Q_input).view(B, L_q, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor hesaplama
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Mask
        if mask is not None:
            mask = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        # Causal mask
        if causal:
            causal_mask = torch.triu(torch.ones(L_q, L_kv, device=Q_input.device), diagonal=1).bool()
            scores = scores.masked_fill(causal_mask, float('-inf'))
        
        # Softmax ve context
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        
        # Head birleştirme
        context = context.transpose(1,2).contiguous().view(B, L_q, E_q)
        
        # Fusion / Gating
        fusion_input = torch.cat([context, Q_input], dim=-1)
        gate = torch.sigmoid(self.gate_proj(fusion_input))
        fused_out = gate * context + (1 - gate) * Q_input
        
        # Residual + LayerNorm
        out = self.layernorm(Q_input + self.dropout(self.out_proj(fused_out)))
        return out, attn

# Test
B, L_q, L_kv, E_q, E_kv = 2, 5, 7, 64, 80
Q_input = torch.randn(B, L_q, E_q)
KV_input = torch.randn(B, L_kv, E_kv)
mask = torch.ones(B, L_kv)

dyn_cross_attn = DynamicFusionCrossAttention(E_q, E_kv, num_heads=8)
y, attn_map = dyn_cross_attn(Q_input, KV_input, mask=mask, causal=True)

print("Output:", y.shape)        # (B, L_q, E_q)
print("Attention map:", attn_map.shape)  # (B, num_heads, L_q, L_kv)


Output: torch.Size([2, 5, 64])
Attention map: torch.Size([2, 8, 5, 7])


## 💡 Avantajlar:

* Q/K/V farklı embedding boyutlarıyla çalışabilir.

* Gating mekanizması sayesinde context ve orijinal Q optimal şekilde birleşir.

- Padding ve causal mask desteği mevcut.

- Multi-head + residual + layernorm + dropout korunuyor.

----

## 🔹 Ultimate Dynamic Cross-Attention

### Özellikler
- Herhangi bir batch, sequence uzunluğu ve embedding boyutunda çalışabilir.  
- Multi-head ile paralel dikkat.  
- Mask ve causal desteği.  
- Opsiyonel gating/fusion ile daha zengin temsil.  
- Residual + LayerNorm + Dropout ile stabil öğrenme.


## Hedefler:

* Dynamic sequence lengths: Girdi boyutları değişken olabilir (L_q ve L_kv).

* Dynamic head / embed_dim: Head sayısı ve embedding boyutu parametre olarak esnek.

* Mask ve causal seçenekleri: Padding ve look-ahead maskeleri otomatik uygulanabilir.

* Gating / Fusion opsiyonel: İstenirse context ve Q fusion kullanılabilir.

* Residual + LayerNorm + Dropout dahil.

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UltimateCrossAttention(nn.Module):
    def __init__(self, embed_dim_q, embed_dim_kv, num_heads=8, dp=0.1, use_gate=True):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim_q // num_heads
        self.use_gate = use_gate
        
        # Projeksiyonlar
        self.q_proj = nn.Linear(embed_dim_q, embed_dim_q)
        self.k_proj = nn.Linear(embed_dim_kv, embed_dim_q)
        self.v_proj = nn.Linear(embed_dim_kv, embed_dim_q)
        self.out_proj = nn.Linear(embed_dim_q, embed_dim_q)
        
        # Gating mekanizması opsiyonel
        if use_gate:
            self.gate_proj = nn.Linear(2*embed_dim_q, embed_dim_q)
        
        self.dropout = nn.Dropout(dp)
        self.layernorm = nn.LayerNorm(embed_dim_q)
    
    def forward(self, Q_input, KV_input, mask=None, causal=False):
        B, L_q, E_q = Q_input.shape
        L_kv = KV_input.size(1)
        
        # Q/K/V projeksiyonları ve head bölünmesi
        Q = self.q_proj(Q_input).view(B, L_q, self.num_heads, self.head_dim).transpose(1,2)
        K = self.k_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        V = self.v_proj(KV_input).view(B, L_kv, self.num_heads, self.head_dim).transpose(1,2)
        
        # Skor
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.head_dim ** 0.5)
        
        # Masking
        if mask is not None:
            mask_exp = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask_exp == 0, float('-inf'))
        if causal:
            causal_mask = torch.triu(torch.ones(L_q, L_kv, device=Q_input.device), diagonal=1).bool()
            scores = scores.masked_fill(causal_mask, float('-inf'))
        
        # Softmax ve context
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        
        # Head birleştirme
        context = context.transpose(1,2).contiguous().view(B, L_q, E_q)
        
        # Fusion / Gating
        if self.use_gate:
            fusion_input = torch.cat([context, Q_input], dim=-1)
            gate = torch.sigmoid(self.gate_proj(fusion_input))
            fused_out = gate * context + (1 - gate) * Q_input
        else:
            fused_out = context
        
        # Residual + LayerNorm + Dropout
        out = self.layernorm(Q_input + self.dropout(self.out_proj(fused_out)))
        return out, attn

# Test
B, L_q, L_kv, E_q, E_kv = 2, 5, 7, 64, 80
Q_input = torch.randn(B, L_q, E_q)
KV_input = torch.randn(B, L_kv, E_kv)
mask = torch.ones(B, L_kv)

ultimate_attn = UltimateCrossAttention(E_q, E_kv, num_heads=8, use_gate=True)
y, attn_map = ultimate_attn(Q_input, KV_input, mask=mask, causal=True)

print("Output:", y.shape)        # (B, L_q, E_q)
print("Attention map:", attn_map.shape)  # (B, num_heads, L_q, L_kv)


Output: torch.Size([2, 5, 64])
Attention map: torch.Size([2, 8, 5, 7])


### 💡 Özellikler:

* Dinamik sequence uzunluğu ve embedding boyutu

* Multi-head paralel dikkat

* Padding ve causal mask desteği

* Opsiyonel gating/fusion ile daha zengin bağlam

* Residual + LayerNorm + Dropout ile stabil öğrenme